Author:
        
        PARK, JunHo, junho@ccnets.org

    COPYRIGHT (c) 2024. CCNets. All Rights reserved.

# Cheat Your Decision-Making Model: Manipulating Credit Card Data with Causal Inference

## Introduction

In this tutorial, we delve into the intricate world of causal inference to manipulate decision-making models in the domain of credit card fraud detection. Utilizing a Cooperative Network (CCNet), we explore how subtle manipulations in non-fraudulent data can expose and test the vulnerabilities of machine learning models. This demonstration not only highlights the robustness and weaknesses of current fraud detection systems but also illustrates the ethical considerations and potential risks inherent in data manipulation.


## Tutorial Goals

This tutorial provides a guide on using causal inference to manipulate data and evaluate its impact on machine learning models. We will cover:

1. **Training a Binary Classifier**: Train a model to distinguish between fraudulent and non-fraudulent transactions using real credit card data.
2. **Generating Manipulated Data with CCNet**: Use CCNet's Explainer, Reasoner, and Producer networks to create data that mimics non-fraudulent transactions but is designed to deceive the classifier by switching decision label factors.
3. **Evaluating Classifier Performance**: Test the classifier on both original and manipulated datasets to assess robustness. The classifier should perform well on both the original test set and the CCNet-recreated test set, indicating that CCNet has learned the data distribution that the classifier mapped for decision making.
4. **Comparative Dataset Analysis**:
   - **Testset A**: Real Test Data - Authentic credit card transactions used as a baseline to evaluate the classifier's performance.
   - **Testset B**: CCNet-generated data with random labels but maintaining the same frequency distribution as the original. The classifier's performance should be similar to Testset A, showing that CCNet understands the classifier's decision-making process.
   - **Testset C**: CCNet-generated data with original labels. With Testset B, it is proved that CCNet creates the genuine data distribution for any case of decision label. However, as most information of the data is in the explanation vector, it is easy to manipulate by changing the decision factor of the fraud data. In short, it is easy to manipulate the decision-making process by causal inference.
5. **Ethical Considerations**: Discuss the ethical implications of data manipulation in machine learning, emphasizing the need for robustness in model training and the risks of misuse in applications like fraud detection.


In [1]:
import sys
path_append = "../"
sys.path.append(path_append)  # Go up one directory from where you are.


In [2]:
# https://fraud-detection-handbook.github.io/fraud-detection-handbook/Chapter_7_DeepLearning/FeedForwardNeuralNetworks.html
import pandas as pd 

dataroot = path_append + "../data/credit_card_fraud_detection/creditcard.csv"
df = pd.read_csv(dataroot)
df

c:\Users\CCNets-team\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\CCNets-team\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [3]:

print('No Frauds', round(df['Class'].value_counts()[0] / len(df) *100,2), '%of the dataset')
print('Frauds', round(df['Class'].value_counts()[1] / len(df) *100,2), '%of the dataset')

No Frauds 99.83 %of the dataset
Frauds 0.17 %of the dataset


In [4]:
import torch
from sklearn.preprocessing import StandardScaler, RobustScaler
from torch.utils.data import Dataset

# Initialize scalers
standard_sc = StandardScaler()
robust_sc = RobustScaler()
# Scale 'Amount' and 'Time' columns separately
df['scaled_amount'] = robust_sc.fit_transform(df['Amount'].values.reshape(-1, 1))
df['scaled_time'] = robust_sc.fit_transform(df['Time'].values.reshape(-1, 1))

# Drop original 'Time' and 'Amount' columns
df.drop(['Time', 'Amount'], axis=1, inplace=True)

# Move the 'Class' column to the end
class_column = df.pop('Class')
df = pd.concat([df, class_column], axis=1)

# Scale the rest of the features (excluding the class column)
df.iloc[:, :-1] = standard_sc.fit_transform(df.iloc[:, :-1])

# Calculate the number of features and classes
n_features = len(df.iloc[:, :-1].columns)
n_classes = len(df.iloc[:, -1:].columns)

print(n_features, n_classes)

30 1


In [5]:
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V22,V23,V24,V25,V26,V27,V28,scaled_amount,scaled_time,Class
0,-0.694242,-0.044075,1.672773,0.973366,-0.245117,0.347068,0.193679,0.082637,0.331128,0.083386,...,0.382854,-0.176911,0.110507,0.246585,-0.392170,0.330892,-0.063781,0.244964,-1.996583,0
1,0.608496,0.161176,0.109797,0.316523,0.043483,-0.061820,-0.063700,0.071253,-0.232494,-0.153350,...,-0.880077,0.162201,-0.561131,0.320694,0.261069,-0.022256,0.044608,-0.342475,-1.996583,0
2,-0.693500,-0.811578,1.169468,0.268231,-0.364572,1.351454,0.639776,0.207373,-1.378675,0.190700,...,1.063358,1.456320,-1.138092,-0.628537,-0.288447,-0.137137,-0.181021,1.160686,-1.996562,0
3,-0.493325,-0.112169,1.182516,-0.609727,-0.007469,0.936150,0.192071,0.316018,-1.262503,-0.050468,...,0.007267,-0.304777,-1.941027,1.241904,-0.460217,0.155396,0.186189,0.140534,-1.996562,0
4,-0.591330,0.531541,1.021412,0.284655,-0.295015,0.071999,0.479302,-0.226510,0.744326,0.691625,...,1.100011,-0.220123,0.233250,-0.395202,1.041611,0.543620,0.651816,-0.073403,-1.996541,0


In [6]:
# Defining the labeled and unlabeled dataset classes
class LabeledDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        vals = torch.tensor(self.x[index], dtype=torch.float32)
        label = torch.tensor(self.y[index], dtype=torch.long)
        return vals, label

#### Dataset Splitting for Training and Testing

The original dataset is split into training and testing parts to evaluate the model's performance accurately. This step is crucial for validating the effectiveness of the training on unseen data.


In [7]:
from sklearn.model_selection import train_test_split

# Splitting the dataset into training and test sets for model evaluation
df_train, df_test = train_test_split(df, test_size=0.5, shuffle=True, random_state=42)
X_train, y_train = df_train.iloc[:, :-1].values, df_train.iloc[:, -1:].values
X_test, y_test = df_test.iloc[:, :-1].values, df_test.iloc[:, -1:].values

# Labeled datasets for supervised learning tasks
trainset = LabeledDataset(X_train, y_train)  # Corrected to include training data
testset = LabeledDataset(X_test, y_test)     # Test set with proper labels

# Printing the shapes of the datasets for verification
print(f"Labeled Trainset Shape: {len(trainset)}, {trainset.x.shape[1]}")
print(f"Labeled Testset Shape: {len(testset)}, {testset.x.shape[1]}")

Labeled Trainset Shape: 142403, 30
Labeled Testset Shape: 142404, 30


#### Initial Setup and Model Configuration

This section initializes the environment by setting a fixed random seed to ensure reproducibility of results. It imports necessary configurations and initializes model parameters with specific configurations. The model specified here is set to have no core model but uses a 'tabnet' encoder model for data processing, which is particularly tailored for structured or tabular data like credit card transactions.


In [8]:
# Set a fixed random seed for reproducibility of experiments
from nn.utils.init import set_random_seed
set_random_seed(0)

# Importing configuration setups for ML parameters and data
import torch
from tools.setting.ml_params import MLParameters
from tools.setting.data_config import DataConfig
from trainer_hub import TrainerHub

# Configuration for the data handling, defining dataset specifics and the task type
data_config = DataConfig(dataset_name='CreditCardFraudDetection', task_type='binary_classification', obs_shape=[n_features], label_size=n_classes, explain_size=n_features - n_classes)

# Initializing ML parameters without a core model and setting the encoder model to 'tabnet' with specific configurations
ml_params = MLParameters(core_model='tabnet', encoder_model='none')

# Setting training parameters and device configuration
ml_params.training.num_epoch = 10
ml_params.model.core_config.num_layers = 3
ml_params.algorithm.error_function = 'mse'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

# Create a TrainerHub instance to manage training and data processing
trainer_hub = TrainerHub(ml_params, data_config, device, use_print=True, use_wandb=False)


c:\Users\CCNets-team\anaconda3\lib\site-packages\transformers\utils\generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [9]:
load_model = False
if load_model:
    trainer_hub.load_trainer(core_model = True)
else:
    trainer_hub.train(trainset)

Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

Iterations:   0%|          | 0/2225 [00:00<?, ?it/s]

[0/10][50/2225][Time 5.52]
Unified LR across all optimizers: 0.0001995308238189185
--------------------Training Metrics--------------------
Cooperative Network(core):  Three Tabnet
Inf: 0.2678	Gen: 0.6190	Rec: 0.6355	E: 0.1346	R: 0.1962	P: 1.4037
[0/10][100/2225][Time 4.87]
Unified LR across all optimizers: 0.00019907191565870155
--------------------Training Metrics--------------------
Cooperative Network(core):  Three Tabnet
Inf: 0.0323	Gen: 0.4207	Rec: 0.4195	E: 0.0191	R: 0.0159	P: 0.9778
[0/10][150/2225][Time 4.79]
Unified LR across all optimizers: 0.00019861406295796434
--------------------Training Metrics--------------------
Cooperative Network(core):  Three Tabnet
Inf: 0.0214	Gen: 0.3725	Rec: 0.3714	E: 0.0147	R: 0.0076	P: 0.7072
[0/10][200/2225][Time 4.91]
Unified LR across all optimizers: 0.00019815726328921765
--------------------Training Metrics--------------------
Cooperative Network(core):  Three Tabnet
Inf: 0.0192	Gen: 0.3554	Rec: 0.3551	E: 0.0104	R: 0.0122	P: 0.7543
[0/10]

Iterations:   0%|          | 0/2225 [00:00<?, ?it/s]

[1/10][25/2225][Time 4.77]
Unified LR across all optimizers: 0.00018030592393534033
--------------------Training Metrics--------------------
Cooperative Network(core):  Three Tabnet
Inf: 0.0027	Gen: 0.2277	Rec: 0.2273	E: 0.0010	R: 0.0001	P: 0.5107
[1/10][75/2225][Time 4.66]
Unified LR across all optimizers: 0.0001798912318178735
--------------------Training Metrics--------------------
Cooperative Network(core):  Three Tabnet
Inf: 0.0038	Gen: 0.2295	Rec: 0.2288	E: 0.0026	R: 0.0004	P: 0.3752
[1/10][125/2225][Time 4.79]
Unified LR across all optimizers: 0.00017947749346581006
--------------------Training Metrics--------------------
Cooperative Network(core):  Three Tabnet
Inf: 0.0048	Gen: 0.2207	Rec: 0.2204	E: 0.0022	R: 0.0014	P: 0.4268
[1/10][175/2225][Time 4.44]
Unified LR across all optimizers: 0.0001790647066855505
--------------------Training Metrics--------------------
Cooperative Network(core):  Three Tabnet
Inf: 0.0029	Gen: 0.2208	Rec: 0.2207	E: 0.0016	R: 0.0008	P: 0.3818
[1/10][2

...

[7/10][25/2225][Time 4.69]
Unified LR across all optimizers: 9.750120782072374e-05
--------------------Training Metrics--------------------
Cooperative Network(core):  Three Tabnet
Inf: 0.0059	Gen: 0.1976	Rec: 0.1986	E: 0.0053	R: 0.0135	P: 0.2825
[7/10][75/2225][Time 4.74]
Unified LR across all optimizers: 9.72769612655121e-05
--------------------Training Metrics--------------------
Cooperative Network(core):  Three Tabnet
Inf: 0.0069	Gen: 0.1922	Rec: 0.1926	E: 0.0090	R: 0.0195	P: 0.2543
[7/10][125/2225][Time 4.85]
Unified LR across all optimizers: 9.705323046306541e-05
--------------------Training Metrics--------------------
Cooperative Network(core):  Three Tabnet
Inf: 0.0034	Gen: 0.1941	Rec: 0.1942	E: 0.0040	R: 0.0040	P: 0.2556
[7/10][175/2225][Time 4.75]
Unified LR across all optimizers: 9.683001422718531e-05
--------------------Training Metrics--------------------
Cooperative Network(core):  Three Tabnet
Inf: 0.0114	Gen: 0.2023	Rec: 0.2025	E: 0.0182	R: 0.0348	P: 0.3196
[7/10][225/

Iterations:   0%|          | 0/2225 [00:00<?, ?it/s]

[8/10][0/2225][Time 4.54]
Unified LR across all optimizers: 8.810691513448475e-05
--------------------Training Metrics--------------------
Cooperative Network(core):  Three Tabnet
Inf: 0.0037	Gen: 0.1833	Rec: 0.1822	E: 0.0088	R: 0.0015	P: 0.1920
[8/10][50/2225][Time 5.16]
Unified LR across all optimizers: 8.790427485288373e-05
--------------------Training Metrics--------------------
Cooperative Network(core):  Three Tabnet
Inf: 0.0025	Gen: 0.1852	Rec: 0.1852	E: 0.0029	R: 0.0035	P: 0.2818
[8/10][100/2225][Time 5.29]
Unified LR across all optimizers: 8.770210063099734e-05
--------------------Training Metrics--------------------
Cooperative Network(core):  Three Tabnet
Inf: 0.0073	Gen: 0.1822	Rec: 0.1827	E: 0.0030	R: 0.0255	P: 0.2068
[8/10][150/2225][Time 4.96]
Unified LR across all optimizers: 8.750039139691806e-05
--------------------Training Metrics--------------------
Cooperative Network(core):  Three Tabnet
Inf: 0.0063	Gen: 0.1794	Rec: 0.1790	E: 0.0030	R: 0.0028	P: 0.1818
[8/10][200/

Iterations:   0%|          | 0/2225 [00:00<?, ?it/s]

[9/10][25/2225][Time 4.78]
Unified LR across all optimizers: 7.943465170874777e-05
--------------------Training Metrics--------------------
Cooperative Network(core):  Three Tabnet
Inf: 0.0057	Gen: 0.1824	Rec: 0.1817	E: 0.0099	R: 0.0041	P: 0.2216
[9/10][75/2225][Time 4.66]
Unified LR across all optimizers: 7.925195707953989e-05
--------------------Training Metrics--------------------
Cooperative Network(core):  Three Tabnet
Inf: 0.0107	Gen: 0.1886	Rec: 0.1899	E: 0.0071	R: 0.0281	P: 0.2470
[9/10][125/2225][Time 4.46]
Unified LR across all optimizers: 7.90696826363192e-05
--------------------Training Metrics--------------------
Cooperative Network(core):  Three Tabnet
Inf: 0.0065	Gen: 0.1823	Rec: 0.1828	E: 0.0070	R: 0.0158	P: 0.2619
[9/10][175/2225][Time 4.83]
Unified LR across all optimizers: 7.888782741268464e-05
--------------------Training Metrics--------------------
Cooperative Network(core):  Three Tabnet
Inf: 0.0048	Gen: 0.1895	Rec: 0.1881	E: 0.0110	R: 0.0018	P: 0.4260
[9/10][225/

In [10]:
# DataLoader for processing training data in larger batches without shuffling
test_loader = torch.utils.data.DataLoader(dataset=testset, batch_size=256, shuffle=False, drop_last=False)

# CCNet setup for generating synthetic manipulated data
ccnet = trainer_hub.core_ccnet
manipulated_data = None
random_labels = None
original_labels = None
probability_positive = 0.0017 # 0.17% of the data is positive

# Generate synthetic data to enhance the diversity of the training dataset
for data, labels in test_loader:
    data = data.to(device)
    labels = labels.to(device)
    
    # Create random labels with 0.17% positive (1) and 99.83% negative (0) labels
    random_label = (torch.rand(labels.size(0)).to(device) < probability_positive).float()
    random_label = torch.nn.functional.one_hot(random_label.to(torch.int64), num_classes=2).float()

    # Use CCNet to explain the original data and generate synthetic counterparts
    explanations = ccnet.explain(data)
    synthetic_data = ccnet.produce(random_label, explanations)
    
    # Detach synthetic data and labels from GPU for further processing
    synthetic_data = synthetic_data.detach().cpu()
    random_label = random_label.detach().cpu().argmax(dim=-1).unsqueeze(-1)
    labels = labels.detach().cpu()

    # Accumulate the generated data for analysis and training
    if manipulated_data is None:
        manipulated_data = synthetic_data
        random_labels = random_label
        original_labels = labels
    else:
        manipulated_data = torch.cat([manipulated_data, synthetic_data], dim=0)
        random_labels = torch.cat([random_labels, random_label], dim=0)
        original_labels = torch.cat([original_labels, labels], dim=0)

# Output the shapes of the datasets for verification
print(f"Manipulated Data Shape: {manipulated_data.shape}")
print(f"Reversed Labels Shape: {random_labels.shape}")
print(f"Original Labels Shape: {original_labels.shape}")

# Create datasets for both synthetic and original label scenarios
random_label_testset = LabeledDataset(manipulated_data.numpy(), random_labels.numpy())
original_label_testset = LabeledDataset(manipulated_data.numpy(), original_labels.numpy())


Manipulated Data Shape: torch.Size([142404, 30])
Reversed Labels Shape: torch.Size([142404, 1])
Original Labels Shape: torch.Size([142404, 1])


#### Training Supervised Models

This section outlines the process of training supervised learning models using both original and synthetic datasets. The `train_supervised_model` function is designed to iterate through the dataset, perform forward passes, compute loss, and update model weights using backpropagation.


In [11]:
def train_supervised_model(model, dataset, num_epoch=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
    # Initialize the optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    # Ensure reproducibility by resetting the random seed
    # Create DataLoader for batch processing
    trainloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)
    # Training loop
    for epoch in range(num_epoch):  # Train for 2 epochs as an example
        for i, (data, label) in enumerate(trainloader):
            data = data.to(device).clone().detach()
            label = label.to(device).float()
            # Perform forward pass
            output = model(data)
            # Compute loss
            loss = torch.nn.functional.binary_cross_entropy(output, label)
            # Backward pass to compute gradients
            loss.backward()
            # Update weights
            optimizer.step()
            # Reset gradients
            optimizer.zero_grad()


In [12]:

class MLP(torch.nn.Module):
    def __init__(self, input_size, output_size, num_layers=4, hidden_size=256):
        super(MLP, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        
        # Create a list to hold all layers
        layers = []
        
        # Input layer
        layers.append(torch.nn.Linear(input_size, hidden_size))
        layers.append(torch.nn.ReLU())
        
        # Hidden layers
        for _ in range(num_layers - 2):
            layers.append(torch.nn.Linear(hidden_size, hidden_size))
            layers.append(torch.nn.ReLU())
        
        # Output layer
        layers.append(torch.nn.Linear(hidden_size, output_size))
        
        # Register all layers
        self.layers = torch.nn.Sequential(*layers)

    def forward(self, x):
        x = self.layers(x)
        return torch.sigmoid(x)
    
# Initialize and train a model on the recreated dataset
decision_making_model = MLP(input_size=n_features, output_size=n_classes).to(device)
train_supervised_model(decision_making_model, trainset)

#### Evaluating Model Performance


In [13]:
from sklearn.metrics import f1_score

def get_f1_score(model, input_testset, device, batch_size=64):
    model.eval()  # Set the model to evaluation mode
    y_true = []
    y_pred = []
    # DataLoader for testing
    test_loader = torch.utils.data.DataLoader(input_testset, batch_size=batch_size, shuffle=False)

    # No gradient computation needed during inference
    with torch.no_grad():
        for data, label in test_loader:
            data = data.to(device)
            label = label.to(device)
            output = model(data)
            # Process output for binary classification
            predicted = (output.squeeze() > 0.5).long()
            y_true.extend(label.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    # Compute and return the F1 score
    score = f1_score(y_true, y_pred, average='binary')
    return score

# Calculate F1 scores for both models
fraud_detection_f1_score = get_f1_score(decision_making_model, testset, device)
random_testset_f1_score = get_f1_score(decision_making_model, random_label_testset, device)
manipulated_testset_f1_score = get_f1_score(decision_making_model, original_label_testset, device)

# Output the results
print("F1 score of the supervised learning model tested on the original data: ", fraud_detection_f1_score)
print("F1 score of the supervised learning model tested on the manipulated data with random label: ", random_testset_f1_score)
print("F1 score of the supervised learning model tested on the manipulated data with original label: ", manipulated_testset_f1_score)

F1 score of the supervised learning model tested on the original data:  0.826271186440678
F1 score of the supervised learning model tested on the manipulated data with random label:  0.7417417417417418
F1 score of the supervised learning model tested on the manipulated data with original label:  0.43609022556390975
